In [1]:
import pandas as pd
import numpy as np
import re
from keras.models import Model
from keras.layers.core import Dropout, Dense, Flatten, Activation
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import GRU
from keras.layers.pooling import MaxPooling2D
from keras.layers import Merge,Input,concatenate


Using TensorFlow backend.


In [2]:
df = pd.read_csv('quora_duplicate_questions.tsv',sep='\t',encoding='utf8')

In [3]:
df = df.dropna()

In [4]:
import spacy
#from gensim.utils import lemmatize
nlp = spacy.load('en')

In [6]:
def clean_data(string):
    #try: 
    rx = re.compile('\W+')
    res = rx.sub(' ',string).strip()
    res = res.replace('\'','')
    return ' '.join([str(x) for x in nlp(res)])
    #return ' '.join([x.lemma_ for x in nlp(res)])

#    return ' '.join([x.lemma_ for x in nlp(res) if not x.is_stop ])

    #except TypeError:
        #print('TypeError',string)

In [7]:
df['q1_clean'] = df['question1'].apply(lambda x: clean_data(x))
df['q2_clean'] = df['question2'].apply(lambda x: clean_data(x))

In [8]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_clean,q2_clean
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,What is the story of Kohinoor Koh i Noor Diamond,What would happen if the Indian government sto...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,Why am I mentally very lonely How can I solve it,Find the remainder when math 23 24 math is div...
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,Which one dissolve in water quikly sugar salt ...,Which fish would survive in salt water


In [9]:
print('q1_counts \n',df['q1_clean'].apply(lambda x: len(x.split())).describe())
print('q2_counts \n',df['q2_clean'].apply(lambda x: len(x.split())).describe())

q1_counts 
 count    404288.000000
mean         11.136019
std           5.581704
min           0.000000
25%           7.000000
50%          10.000000
75%          13.000000
max         128.000000
Name: q1_clean, dtype: float64
q2_counts 
 count    404288.000000
mean         11.384572
std           6.490109
min           0.000000
25%           7.000000
50%          10.000000
75%          13.000000
max         248.000000
Name: q2_clean, dtype: float64


In [10]:
def build_vocab(string):
    for x in string.split():
        vocab.add(x)
        lexicon.append(x)
    return

In [11]:
vocab = set()
lexicon = []

In [12]:
_ = df['q1_clean'].apply(lambda x: build_vocab(x))
_ = df['q2_clean'].apply(lambda x: build_vocab(x))

In [13]:
vocab_size = len(vocab)

In [14]:
vocab_size

109244

In [15]:
def generate_labels(inputs, cnter,max_len,vocab_len):
    np_input = []
    
    for sentences in inputs:
        sen  = np.full(max_len,vocab_len)
        for idx,word in enumerate(sentences.split()):
            if(idx<max_len):
                sen[idx] = cnter.index(word)
        np_input.append(sen)
        
    return np.array(np_input).astype(np.int)
        
        

In [16]:
from collections import Counter
vocab_cnter = Counter(lexicon)

In [17]:
sorted_vocab = [x[0] for x in vocab_cnter.most_common()]

In [18]:
sorted_vocab[:10]

['the', 'What', 'I', 'is', 'a', 'to', 'How', 'in', 'of', 'do']

In [19]:
word2vec = []
for word in sorted_vocab:
    word2vec.append(nlp(word).vector)

In [20]:
len(word2vec)

109244

In [21]:
word2vec = np.array(word2vec)
word2vec.shape

(109244, 300)

In [22]:
q1_inputs = generate_labels(df['q1_clean'],sorted_vocab,30,len(vocab))
print('q1 processed')
q2_inputs = generate_labels(df['q2_clean'],sorted_vocab,30,len(vocab))

q1 processed


In [68]:
from keras.models import Model
from keras.layers.core import  Lambda,Dropout,Dense, Flatten, Activation
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.recurrent import GRU,LSTM
from keras.layers.pooling import MaxPooling2D
from keras.layers import Concatenate, Input, concatenate, BatchNormalization
from keras import backend as K
def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

# Q1
input_1 = Input(shape=(30,))
embedd = Embedding(len(sorted_vocab),300,
                        weights =[word2vec],
                        name="Embedding_1")
embedding_1 = (embedd)(input_1)

bi_gru_1 = Bidirectional(LSTM(200,name='LSTM_1',dropout=.8,return_sequences=True))(embedding_1)
bi_gru_1_2 =  Bidirectional(LSTM(100,name="LSTM_1_2",dropout=.8,recurrent_dropout=.8,return_sequences=True))(bi_gru_1)
bi_gru_1_3 =  Bidirectional(LSTM(50,name="LSTM_1_2",dropout=.8,recurrent_dropout=.8))(bi_gru_1_2)
#Q2 
input_2 = Input(shape=(30,))
embedding_2 = (embedd)(input_2)
bi_gru_2 = Bidirectional(LSTM(200,name="LSTM_2",dropout=.8,recurrent_dropout=.8,return_sequences=True))(embedding_2)
bi_gru_2_2 =  Bidirectional(LSTM(100,name="LSTM_2_2",recurrent_dropout=.8,return_sequences=True))(bi_gru_2)
bi_gru_2_3 = Bidirectional(LSTM(50,name='LSTM_2_3'))(bi_gru_2_2)

#Merge
#cosine distance
merged_1 = Lambda(cosine_distance,output_shape=cos_dist_output_shape)([bi_gru_2_3,bi_gru_1_3])
#euclid distance
#merged_1 = Flatten()(merged_1)
#print(merged_1)
merged_2 = Lambda(euclidean_distance,output_shape=eucl_dist_output_shape)([bi_gru_2_3,bi_gru_1_3])
#merged_2 = Flatten()(merged_2)
merged = concatenate([merged_1,merged_2])
#merged = (Flatten())(merged)
#Dense
dense = Dense(4098,activation='relu',name='dense')(merged)
drop_1 = Dropout(.5,name='drop_1')(dense)
dense_2 = Dense(1000,activation='relu',name='dense_2')(drop_1)
drop_2 = Dropout(.5,name='drop_2')(dense_2)
pred = Dense(2,activation="softmax",name='pred')(drop_2)
model = Model(inputs=[input_1,input_2],outputs=pred)
# model = Sequential()
# model.add(Embedding(len(vocab),64,input_length=sentence_len,name='Embedding_Input'))
# model.add(Bidirectional(GRU(100,dropout=.5,name='GRU')))
# model.add(Dense(1000,activation='relu',))

In [26]:
q1_inputs.shape

(404288, 30)

In [27]:
q2_inputs.shape

(404288, 30)

In [61]:
def f1_score(tags, predicted):
    #print(tags)
    #print(K.softmax(predicted))
    tags = set(tags)
    #print(tags)
    predicted = set(K.(predicted))

    tp = len(tags & predicted)
    fp = len(predicted) - tp 
    fn = len(tags) - tp

    if tp>0:
        precision=float(tp)/(tp+fp)
        recall=float(tp)/(tp+fn)
        return 2*((precision*recall)/(precision+recall))
    else:
        return 0

In [63]:

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


In [73]:
from keras import optimizers
import keras
from sklearn import metrics
adam = optimizers.Adam(lr=.0001)
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph_2_lstm_with_f1', histogram_freq=0, write_graph=True, write_images=True)
model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])
#model.fit(x=[q1_inputs,q2_inputs],y=pd.get_dummies(df['is_duplicate']).values,
          #batch_size=64,epochs=100,validation_split=.1,callbacks=[tbCallBack])
train_loop(model,50,[q1_inputs,q2_inputs],pd.get_dummies(df['is_duplicate'].values),None)

ValueError: Found input variables with inconsistent numbers of samples: [2, 404288]

In [72]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
def train_loop(model, epochs,feats,labels,callbacks):
    x_train,x_test,y_train,y_test = train_test_split(feats,labels,test_size=.1,random_state=4)
    for i in range(epochs):
        model.fit(x=x_train,y=y_train, batch_size=64,ephocs=1)
        print('F1:',metrics.f1_score(y_test,(model.predict(x_test))))

    model.save('LSTM_3.p')
    